In [2]:
import torch

In [ ]:
# Создаем входной массив из двух изображений RGB 3*3
input_images = torch.tensor(
      [[[[0,  1,  2],
         [3,  4,  5],
         [6,  7,  8]],

        [[9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]],

        [[18, 19, 20],
         [21, 22, 23],
         [24, 25, 26]]],


       [[[27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44]],

        [[45, 46, 47],
         [48, 49, 50],
         [51, 52, 53]]]])


def get_padding2d(input_images):
    #padded_images = # добавить нулей с четырех сторон каждого изображения
    ## Top button
    t=list(input_images.size())
    t[2]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 2)
    input_images=torch.cat((input_images,newrow), 2)
    input_images


    ## Sides
    t=list(input_images.size())
    t[-1]=1
    newrow=torch.zeros(t)
    newrow=newrow.type(torch.long)
    input_images=input_images.type(torch.long)
    
    input_images=torch.cat((newrow, input_images), 3)
    input_images=torch.cat((input_images, newrow), 3)
    padded_images=input_images
    
    padded_images=padded_images.type(torch.float)
    return padded_images

In [3]:
get_padding2d(input_images)

tensor([[[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  2.,  0.],
          [ 0.,  3.,  4.,  5.,  0.],
          [ 0.,  6.,  7.,  8.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  9., 10., 11.,  0.],
          [ 0., 12., 13., 14.,  0.],
          [ 0., 15., 16., 17.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 18., 19., 20.,  0.],
          [ 0., 21., 22., 23.,  0.],
          [ 0., 24., 25., 26.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 27., 28., 29.,  0.],
          [ 0., 30., 31., 32.,  0.],
          [ 0., 33., 34., 35.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 36., 37., 38.,  0.],
          [ 0., 39., 40., 41.,  0.],
          [ 0., 42., 43., 44.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  0.,  0.],
          [ 0., 45., 46., 

## My Conv2d

In [45]:
import torch
from abc import ABC, abstractmethod


def calc_out_shape(input_matrix_shape, out_channels, kernel_size, stride, padding):
    batch_size, channels_count, input_height, input_width = input_matrix_shape
    output_height = (input_height + 2 * padding - (kernel_size - 1) - 1) // stride + 1
    output_width = (input_width + 2 * padding - (kernel_size - 1) - 1) // stride + 1

    return batch_size, out_channels, output_height, output_width


class ABCConv2d(ABC):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

    def set_kernel(self, kernel):
        self.kernel = kernel

    @abstractmethod
    def __call__(self, input_tensor):
        pass


In [51]:
# Сверточный слой через циклы.
class Conv2dLoop(ABCConv2d):
    def __call__(self, input_tensor):
        # output_tensor = # Напишите в этом месте вычисление свертки с использованием циклов.

        # проверка является ли вход батчем ( dimension = 4 => batch, dimension = 3 => one_image)
        # if len(input_tensor.shape) = 4:
            # for i in input_tensor.shape[0]:
        return output_tensor

In [52]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

out_channels = kernel.shape[0]
in_channels  = kernel.shape[1]
kernel_size  = kernel.shape[2]
stride=2

print(f" \
out_channels {out_channels} \n \
in_channels  {in_channels}  \n \
kernel_size  {kernel_size}  \n \
"
)

 out_channels 1 
 in_channels  3  
 kernel_size  3  
 


In [58]:
my_conv2d = Conv2dLoop(in_channels, out_channels, kernel_size, stride)
my_conv2d.set_kernel(kernel)

In [56]:
input_tensor.shape[0]

2

In [37]:
len(input_tensor.shape)

4

In [ ]:
## Torch conv2d

In [33]:
a = torch.randint(10, (1, 3, 7, 7)).to(torch.float32)
a

tensor([[[[0., 1., 8., 3., 0., 6., 0.],
          [5., 4., 8., 6., 8., 2., 0.],
          [8., 6., 3., 4., 7., 0., 5.],
          [0., 5., 2., 4., 0., 6., 0.],
          [1., 8., 1., 7., 6., 5., 8.],
          [8., 6., 2., 0., 1., 1., 8.],
          [6., 7., 4., 0., 6., 8., 0.]],

         [[9., 5., 8., 7., 4., 0., 1.],
          [4., 8., 2., 6., 2., 3., 5.],
          [3., 9., 6., 4., 8., 6., 7.],
          [5., 3., 8., 4., 8., 6., 6.],
          [1., 5., 3., 2., 9., 9., 3.],
          [1., 4., 6., 3., 1., 5., 6.],
          [9., 6., 7., 3., 4., 2., 3.]],

         [[7., 4., 9., 6., 7., 9., 5.],
          [9., 1., 2., 8., 9., 3., 4.],
          [2., 2., 4., 7., 9., 2., 7.],
          [0., 9., 1., 4., 0., 2., 6.],
          [7., 1., 9., 5., 4., 2., 9.],
          [5., 5., 4., 0., 2., 3., 6.],
          [2., 1., 1., 4., 6., 8., 7.]]]])

In [34]:
class Conv2d(ABCConv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        self.conv2d = torch.nn.Conv2d(in_channels, out_channels, kernel_size,
                                      stride, padding=0, bias=False)

    def set_kernel(self, kernel):
        self.conv2d.weight.data = kernel

    def __call__(self, input_tensor):
        return self.conv2d(input_tensor)

In [35]:
torch_conv2d = Conv2d(3,4,3,1)
torch_conv2d(a).shape

torch.Size([1, 4, 5, 5])

In [ ]:
## Test input 

In [22]:
kernel = torch.tensor(
                  [[[[0., 1, 0],
                     [1,  2, 1],
                     [0,  1, 0]],

                    [[1, 2, 1],
                     [0, 3, 3],
                     [0, 1, 10]],

                    [[10, 11, 12],
                     [13, 14, 15],
                     [16, 17, 18]]]])

in_channels = kernel.shape[1]
in_channels

3

In [24]:
batch_size

(2,)

In [25]:
batch_size=2


In [29]:
batch_size=2
input_height=4
input_width=4
stride=2

print(batch_size * in_channels * \
                            input_height * input_width)

input_tensor = torch.arange(0, batch_size * in_channels *
                            input_height * input_width,
                            out=torch.FloatTensor()) \
    .reshape(batch_size, in_channels, input_height, input_width)
print(input_tensor.shape)
input_tensor

96
torch.Size([2, 3, 4, 4])


tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[16., 17., 18., 19.],
          [20., 21., 22., 23.],
          [24., 25., 26., 27.],
          [28., 29., 30., 31.]],

         [[32., 33., 34., 35.],
          [36., 37., 38., 39.],
          [40., 41., 42., 43.],
          [44., 45., 46., 47.]]],


        [[[48., 49., 50., 51.],
          [52., 53., 54., 55.],
          [56., 57., 58., 59.],
          [60., 61., 62., 63.]],

         [[64., 65., 66., 67.],
          [68., 69., 70., 71.],
          [72., 73., 74., 75.],
          [76., 77., 78., 79.]],

         [[80., 81., 82., 83.],
          [84., 85., 86., 87.],
          [88., 89., 90., 91.],
          [92., 93., 94., 95.]]]])